<a href="https://colab.research.google.com/github/AnaliaLeyez/Clutch-game/blob/main/u3_limpieza_parte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UNIDAD III - Limpieza y preparación

El notebook esta basado en el contenido del libro Python for Data Analysis.
[Chapter 7 Data Cleaning and Preparation. (Wes McKinney)](https://wesmckinney.com/book/data-cleaning)



# Inicializaciones

## Importamos dependencias

In [ ]:
import numpy as np
import pandas as pd


### Parametros por defecto

In [ ]:
np.random.seed(12345)

# Carga y diagnóstico base

In [ ]:
# Dataset sintético para practica
practica = pd.DataFrame({
    'id':[1,1,2,3,4,5],
    'nombre':[' Ana ', 'ana', 'TOM', 'Luz', None, '   Marta'],
    'monto':['1.234,00','500,50', np.nan, '100,0', '10.000,99', 'quince'],
    'categoria':['A','a','B','B','C','C '],
    'fecha':['2024-01-10','2024/01/11','2055-01-01', None, '01-02-2024', '2024-02-03'],
})
practica

,id,nombre,monto,categoria,fecha
0,1,Ana,"1.234,00",A,2024-01-10
1,1,ana,"500,50",a,2024/01/11
2,2,TOM,NaN,B,2055-01-01
3,3,Luz,"100,0",B,None
4,4,None,"10.000,99",C,01-02-2024
5,5,Marta,quince,C,2024-02-03


## Tamaño del dataset

In [ ]:
# Nro de filas y columnas)
practica.shape

(6, 5)

## Estructura del dataset

In [ ]:
#Resumen estándar
practica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         6 non-null      int64 
 1   nombre     5 non-null      object
 2   monto      5 non-null      object
 3   categoria  6 non-null      object
 4   fecha      5 non-null      object
dtypes: int64(1), object(4)
memory usage: 372.0+ bytes


## Estadísticas descriptivas

In [ ]:
#(count:cantidad de valores no nulos),mean: media o promedio.std: desviación estándar (dispersión).
# min, quartiles, max) de columnas numéricas para ver rangos/outliers
practica.describe ()

,id
count,6.000000
mean,2.666667
std,1.632993
min,1.000000
25%,1.250000
50%,2.500000
75%,3.750000
max,5.000000


## 2) **Tipificación**: convertir tipos (números/fechas)

Muchos datasets vienen con formato local distinto al que utilizamos (puntos de miles y coma decimal) y fechas como texto. Si no tipificamos, puede ocurrir que Pandas trate esos campos como strings. Por lo cual el objetivo de la tipificacion es llevar cada columna a su tipo correcto (numérico/fecha) para poder analizar sin errores y con buena performance.

In [ ]:
## trabajamos sobre una copia para conservar el original
df = practica.copy()

# 1) Normalizar números en formato “AR/ES” (miles='.', decimales=',') → formato estándar con punto decimal
# astype(str) pone toda la columna en formato texto para poder usar str de forma segura y vectorizada
df['monto'] = (df['monto'].astype(str)
                .str.replace('.', '', regex=False)  # quita separadores de miles
                .str.replace(',', '.', regex=False))# convierte la coma decimal a punto

# 2) Convertir a numérico y los valores inválidos a NaN
df['monto'] = pd.to_numeric(df['monto'], errors='coerce')

# 3) Parsear fechas desde texto (día/mes primero) a datetime y los valores inválidos a NaT
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce', dayfirst=True)

# 4) Ajustar tipos “óptimos” de pandas (Int64, string, boolean, etc.)
df = df.convert_dtypes()

df.dtypes  # verificación: confirma tipos finales por columna

,0
id,Int64
nombre,string[python]
monto,Float64
categoria,string[python]
fecha,datetime64[ns]


# Datos faltantes / perdidos

¿Por qué importan y cómo los tratamos?

En datasets reales es habitual que falten valores (no medidos, errores de carga, formatos inconsistentes). Ignorarlos distorsiona análisis y rompe modelos.

Por ello nuestro objetivo es detectar, hacer visibles los faltantes (muchas veces aparecen recién al tipificar números/fechas) y decidir un tratamiento coherente con el problema.

Causas típicas:

- No se midió / no aplica.

- Error de captura o formato (“quince”, fecha inválida).

- Integración de fuentes heterogéneas.

¿Còmo tratarlos?:

1-Diagnosticar.

2-Tipificar: convertir números y fechas para revelar NA ocultos.

3-Decidir: imputar (mediana/0/forward fill), dejar NA o descartar filas/columnas según impacto.

4-Validar: revisar estadísticas antes/después

5-Documentar: anotar por qué se eligió cada criterio.

**Importante hay que decidir con criterio, no “por defecto”; diferenciá el cero de desconocido y conservá una versión original para comparar los cambios.**


`isna()` retorna una serie booleana con True donde hay valores nulos (`NaN` o `None`)

In [ ]:
# ¿Dónde faltan datos?
practica.isna()

,id,nombre,monto,categoria,fecha
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,True,False,False
3,False,False,False,False,True
4,False,True,False,False,False
5,False,False,False,False,False


In [ ]:
  # ¿Dónde faltan datos? máscara booleana fila a fila
practica['fecha'].isna()

,fecha
0,False
1,False
2,False
3,True
4,False
5,False


In [ ]:
# ¿Cuantos por columna?
practica.isna().sum()

,0
id,0
nombre,1
monto,1
categoria,0
fecha,1


In [ ]:
# ¿Que % por columna?
practica[['nombre','monto','fecha']].isna().mean().round(2)

,0
nombre,0.17
monto,0.17
fecha,0.17


In [ ]:
# ¿que columnas tinen todos NA?
practica.isna().all()

,0
id,False
nombre,False
monto,False
categoria,False
fecha,False


In [ ]:
# que columnas tienen al menos algun NA
practica.isna().any()

,0
id,False
nombre,True
monto,True
categoria,False
fecha,True


In [ ]:
practica

,id,nombre,monto,categoria,fecha
0,1,Ana,"1.234,00",A,2024-01-10
1,1,ana,"500,50",a,2024/01/11
2,2,TOM,NaN,B,2055-01-01
3,3,Luz,"100,0",B,None
4,4,None,"10.000,99",C,01-02-2024
5,5,Marta,quince,C,2024-02-03


## Como encontramos valores NA

In [ ]:
# que columnas tienen al menos algun NA
practica.isna().any()

,0
id,False
nombre,True
monto,True
categoria,False
fecha,True


In [ ]:
#que filas tienen todos NA
practica.isna().all(axis=1)

,0
0,False
1,False
2,False
3,False
4,False
5,False


In [ ]:
# que fila tienen al menos algun NA
practica.isna().any(axis=1)

,0
0,False
1,False
2,True
3,True
4,True
5,False


In [ ]:
# cuantos na hay en cada fila
practica.isna().sum(axis="columns")

,0
0,0
1,0
2,1
3,1
4,1
5,0


## Filtrar datos perdidos (dropna)

Existe dos metodos utiles para manipular datos perdidos o no disponibles

* dropna() -> sirve para **quitar** datos cuando hay datos "no disponibles" (NA o perdidos)
* fillna() -> sirve para **completar** cuando hay datos perdidos

`dropna` retorna un nuevo dataframe filtrando los valores null

In [ ]:
practica.dropna()

,id,nombre,monto,categoria,fecha
0,1,Ana,"1.234,00",A,2024-01-10
1,1,ana,"500,50",a,2024/01/11
5,5,Marta,quince,C,2024-02-03


Existen muchas opciones para controlar el comportamiento de [`dropna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) en series y en dataframe

In [ ]:
# armamos un dataset con varios datos NA
data = pd.DataFrame([[1., 6.5, 3.],
                     [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan],
                     [np.nan, 6.5, 3.]])

data.columns = ["A", "B", "C"]

data

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [ ]:
# quita todos los renglones con algun NA (not available)
data.dropna()

,A,B,C
0,1.0,6.5,3.0


Con `how` definimos como quitar.
* "any" quita la fila si al menos hay algun NA. (default)
* "all" quita solo si todos son NA

In [ ]:
data.dropna(how="all")

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Parametro `axis` indica que eliminar.

* 'index' o 0 : remueve renglones
* 'columns' o 1 : remueve columnas.

In [ ]:
# agregamos una columna toda NA
data["D"] = np.nan

# mostramos
data

,A,B,C,D
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [ ]:
# removemos la unica columna que tiene todos sus valores NA
data.dropna(axis="columns", how="all")

,A,B,C
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


El parámetro `tresh` indica desde cuando opera `dropna`. `dropna` funciona cuando la cantidad de NA son mayor o igual a `tresh`

In [ ]:
#Df para ejemplo
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
# agregamos algunos NaN
df.iloc[0:4, 1] = np.nan
df.iloc[0:2, 2] = np.nan
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
# quita cuando hay dos o mas Nan
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
# dropear cuando hay nan en la serie 2
df.dropna(subset=[2])

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


## Llenar datos perdidos (fillna)

fillna se puede usar para llenar / completar los datos perdidos.

In [ ]:
# armamos dataframe de ejemplo
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.476985,NaN,NaN
1,-0.577087,NaN,NaN
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


`fillna` retorna un DataFrame llenando cualquier valor NA con el valor especificado

In [ ]:
df.fillna(0)

,0,1,2
0,0.476985,0.000000,0.000000
1,-0.577087,0.000000,0.000000
2,0.523772,0.000000,1.343810
3,-0.713544,0.000000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


Podemos hacer que funcione con ciertos valores para cada columna

In [ ]:
# usar 0.5 para la segunda columna y 0.0 para la última
df.fillna({1: 0.5, 2: 7})

,0,1,2
0,0.476985,0.500000,7.000000
1,-0.577087,0.500000,7.000000
2,0.523772,0.500000,1.343810
3,-0.713544,0.500000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


In [ ]:
# renobramos las columnas
df.columns = ["A", "B", "C"]
df

,A,B,C
0,0.476985,NaN,NaN
1,-0.577087,NaN,NaN
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


In [ ]:
# Lo mas comun es tener series con nombres de strings
df.fillna({"B": 0.5, "C": 0})

,A,B,C
0,0.476985,0.500000,0.000000
1,-0.577087,0.500000,0.000000
2,0.523772,0.500000,1.343810
3,-0.713544,0.500000,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512
6,0.332883,-2.359419,-0.199543


El parámetro `method` permite definir con que valores completa.
* `ffill` completa el valor con el último valor del eje 'yendo hacia adelante'.
* `bfill` completa el valor con el último valor del eje 'yendo hacia atras'.


In [ ]:
# Creamos Df de ejemplo y le agregamos NaN
df = pd.DataFrame(np.random.standard_normal((6, 3)), columns=["A", "B", "C"])
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,A,B,C
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,NaN,0.069877
3,0.246674,NaN,1.004812
4,1.327195,NaN,NaN
5,0.022185,NaN,NaN


In [ ]:
#completa el valor con el último valor del eje 'yendo hacia adelante'.
df.ffill()

,A,B,C
0,-1.541996,-0.970736,-1.307030
1,0.286350,0.377984,-0.753887
2,0.331286,0.377984,0.069877
3,0.246674,0.377984,1.004812
4,1.327195,0.377984,1.004812
5,0.022185,0.377984,1.004812


Podemos ffill y bfill para completar datos de solo ciertas columnas usando indexación de los dataframe

In [ ]:
df["B"]

,B
0,-0.970736
1,0.377984
2,NaN
3,NaN
4,NaN
5,NaN


In [ ]:
#Completamos varias columnas (por columna, vertical)
df[["B","C"]].ffill()

,B,C
0,-0.970736,-1.307030
1,0.377984,-0.753887
2,0.377984,0.069877
3,0.377984,1.004812
4,0.377984,1.004812
5,0.377984,1.004812


Podemos usar los valores estadísticos para completar los NAs.

Por ejemplo, completar los NA de una `Serie`| con su **media**.

In [ ]:
#creamos serie para el ejemplo
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

,0
0,1.0
1,NaN
2,3.5
3,NaN
4,7.0


In [ ]:
#Rellena los NaN con la media de la serie (la media se calcula ignorando los NaN).
data.fillna(data.mean())

,0
0,1.000000
1,3.833333
2,3.500000
3,3.833333
4,7.000000


## Transformar con inplace

In [ ]:
# armamos dataframe de ejemplo
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,0.862580,-0.010032,0.050009
1,0.670216,0.852965,-0.955869
2,-0.023493,NaN,-0.652469
3,-1.218302,NaN,1.074623
4,0.723642,NaN,NaN
5,-0.503087,NaN,NaN


In [ ]:
#Eliminar filas con 2 valores NO nulos
df.dropna(thresh=2, inplace=True)
df

,0,1,2
0,0.862580,-0.010032,0.050009
1,0.670216,0.852965,-0.955869
2,-0.023493,NaN,-0.652469
3,-1.218302,NaN,1.074623


# Transformaciones

## Quitar duplicados

A continuación un conjunto de funciones de pandas utiles para poder transformar los datos con el fin de limpiarlos y preparlos.

In [ ]:
# DataFrame de ejemplo
df = pd.DataFrame({
    "k1": ["A","A","B","B","B","C"],
    "k2": [ 1 , 1 , 1 , 2 , 2 , 3 ]
})
print(df)

  k1  k2
0  A   1
1  A   1
2  B   1
3  B   2
4  B   2
5  C   3


El método `duplicate` sirve para encontrar renglones repetidos que tengan valores repetidos en todas o algunas columnas. Si hay varios duplicados, mantiene el primero y marca el resto como duplicados.

In [ ]:
df.duplicated()
#devuelve una Serie booleana: False en la 1ª aparición, True en repeticiones

,0
0,False
1,True
2,False
3,False
4,True
5,False


`drop_duplicates` sirve para elimnar los duplicados. Si hay varios duplicados, mantiene el primero y elimina el resto como duplicados.

In [ ]:
df.drop_duplicates()

,k1,k2
0,A,1
2,B,1
3,B,2
5,C,3


Podemos elegir quedarnos con los últimos duplicados

In [ ]:
df.drop_duplicates(subset=["k1"], keep='last')

,k1,k2
1,A,1
4,B,2
5,C,3


## Quitar ejes

Método `drop`


In [ ]:
k1 = ["one", "two"] * 3 + ["two"]
k2 = [1, 1, 2, 3, 3, 4, 4]
v1 = [0, 1, 2, 3, 4, 5, 6]

data = pd.DataFrame({"k1": k1,
                     "k2": k2,
                     "v1": v1 })
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


Eliminar filas por su `index`

In [ ]:
data.drop(index=[0,2,4,6])

,k1,k2,v1
1,two,1,1
3,two,3,3
5,two,4,5


Eliminar columnas

In [ ]:
data.drop(columns=["v1"])

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


## Remplazar valores

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3. ,2, 4, 7])
data

,0
0,1.0
1,-999.0
2,2.0
3,-999.0
4,-1000.0
5,3.0
6,2.0
7,4.0
8,7.0


Otra manera de realizar transformaciones es con [`replace`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html)

Supongamos que detectamos valores extremos que no corresponde a una medición con sentido.

In [ ]:
# podemos reemplazar de aun valor
data.replace(-999, np.nan)

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,-1000.0
5,3.0
6,2.0
7,4.0
8,7.0


In [ ]:
# si pasamos una lista podemos reemplazar varios valores en unico valor
data.replace([-999, -1000], np.nan)

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,NaN
5,3.0
6,2.0
7,4.0
8,7.0


In [ ]:
# podemos reemplazar varios valores y asignar un con una lista el valor
# por el cual reemplazar
data.replace([-999, -1000], [np.nan, 0])

,0
0,1.0
1,NaN
2,2.0
3,NaN
4,0.0
5,3.0
6,2.0
7,4.0
8,7.0


## Renombrar indices y columnas

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Podemos cambiarlo obteniendo el indice y luego modificarlo

In [ ]:
# Este es el indice original
data.index

Index(['Ohio', 'Colorado', 'New York'], dtype='object')

In [ ]:
def transform(x):
    return x[:4].upper() #toma los 4 primeros caracteres y los convierte a MAYÚSCULAS

data.index.map(transform) #aplica 'transform' a cada etiqueta del índice y devuelve un NUEVO Index

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [ ]:
# map retorna un nuevo index. Ahora actualizamos el nuevo indice
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Con `rename` podemos cambiar los indices y las columnas de manera más concisa y flexible. Ver documentación [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)  

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [ ]:
# o podemos pasar un diccionario
data.rename(index={"Ohio": "Indiana"},
           o" columns={"three": "peekabo})

,one,two,peekaboo,four
Indiana,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [ ]:
# podemos pasar funciones para que realicen el cambio
data.rename(index=str.title, columns=str.upper)
#El index se transforman con str.title (cada palabra en Title, 1ra letra mayuscula).
#Las columnas se transforman con str.upper (MAYÚSCULAS).

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


`rename` no modifica los valores del dataframe, para eso podemos usar la opcion `inplace=True`

In [ ]:
data.rename(index=str.title, columns=str.upper, inplace=True)
data

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11
